## **Data analyisis example**

In [1]:
from db_adapter import *
import pandas as pd

#### **Setting up the connection**

Boto3 will automatically look for credentials and default region in the AWS CLI confinguration file. More information [here](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html)

In [2]:
conn = Connection()

#### Accessing tables

The method get_tables() returns a name to table dictionary with all the tables on a region. If you know what table you're looking for, the method get_table can be called with the corresponding name. Note that the tables here are only references to their remote counterparts.

In [3]:
tables = conn.get_tables()
for name, table in tables.items():
    print(name)

Clientes
Fornecedores
Fornecedores_Produtos
Lojas
Produtos
Vendas
Vendas_Produtos


#### Loading tables to memory

You can load the contents of a table by performing an unfiltered scan. Both scan() and query() methods return a lais of dictionaries that can be passed directly to the pandas.DataFrame() constructor. If the attribute schema is inconsistent, that may cause problems.

In [4]:
dfs = {name: table.scan() for name, table in tables.items()}

In [5]:
dfs['Vendas_Produtos'].index.get_level_values('produto')

Index([0, 2, 5, 1, 4, 3], dtype='object', name='produto')

In [6]:
dfs['Produtos']

,nome,descricao
codigo,,
3,Pepino,Pepino!
2,Água em Pó,Basta colocar água!
4,Mineirinho Adventures,Nada a comentar!
1,Vaporizador RGB,Infinitas opções de customização!
0,Supositório Gamer,Cague como um campeão!
5,Dick Augmentator Tabajara,Aumente seu pepino agora mesmo!


In [7]:
dfs['Vendas']

,,cliente,data
loja,num_nota_fiscal,,
3,1,3,2023-08-18
2,3,4,2023-01-20
1,2,2,2023-03-03
0,0,1,2023-06-14


In [8]:
dfs['Clientes']

,endereco,nome
codigo,,
3,Fortaleza,Roberto
2,Recife,Carlos
4,Porto Alegre,Maria
1,São Paulo,Antônio
0,Salvador,Raíssa


#### Querying tables

Tables in DynamoDB can be queried in two ways:
1. A **scan**, where the engine iterates through the whole table. This is the slower but more flexible method
2. A **query**, where the engine iterates through one partition and key range only. This is more restrictive but much faster.

**Let's try analyzing Maria's purchases:**

In [9]:
tables['Clientes'].key_schema()

{'HASH': 'codigo'}

First, we get ger client ID.

> Note that if *nome* was the partition key, with *codigo* being the range key, the scan below would not be necessary. This was a database design mistake on my part.

In [10]:

maria_id = tables['Clientes'].scan(Attr('nome').eq('Maria'), as_df=False)[0]['codigo']
maria_id

Decimal('4')

Next, we scan the *Vendas* (sales) table to look for the purchases linked to Maria's ID.

In [11]:
maria_sales = tables['Vendas'].scan(Attr('cliente').eq(maria_id), as_df=False)
maria_sales

[{'loja': Decimal('2'),
  'num_nota_fiscal': Decimal('3'),
  'cliente': Decimal('4'),
  'data': ' 2023-01-20'}]

How lucky of us! There's only one matching purchase. It's now possible to run a quick query to get the product list from *Vendas_Produtos* (sales_products):

In [12]:
sale_details = tables['Vendas_Produtos'].query(Key('venda').eq(maria_sales[0]['num_nota_fiscal']))
#sale_details = pd.DataFrame(sale_details)
sale_details

preco quantidade
venda produto                 
3     0          50         50
      2          20         10

The code below queries the *Produtos* (products) table to get the list of products from Maria's sales. This avoids the whole table to memory.

In [13]:
maria_products = set(sale_details.index.get_level_values('produto'))
maria_products = [tables['Produtos'].get_item({'codigo': product}) for product in maria_products]
maria_products = to_df(maria_products, {'HASH': 'codigo'})
maria_products

,nome,descricao
codigo,,
0,Supositório Gamer,Cague como um campeão!
2,Água em Pó,Basta colocar água!


The final table is generated with the following code:

In [14]:
final_df = pd.merge(
    sale_details, 
    maria_products,
    left_on='produto',
    right_on='codigo'
)
final_df = final_df[['quantidade', 'nome', 'descricao', 'preco']]
final_df['subtotal'] = final_df['quantidade'] * final_df['preco']
final_df.rename(
    axis='columns',
    inplace=True,
    mapper={
        'quantidade': 'quantity',
        'nome': 'product_name',
        'descricao': 'description',
        'preco': 'price',
    }
) 
final_df

,quantity,product_name,description,price,subtotal
0,50,Supositório Gamer,Cague como um campeão!,50,2500
1,10,Água em Pó,Basta colocar água!,20,200


#### Querying indices

Remember the mistake I made with the *Clientes* table? We can sort of fix that by using a Global Secondary Index (GSI). To do so, we must first know what indices with what key schemas are avaliable to us.

In [15]:
gsi_s = tables['Clientes'].secondary_indices()['GLOBAL']
gsi_s

[{'IndexName': 'nome-codigo-index',
  'KeySchema': [{'AttributeName': 'nome', 'KeyType': 'HASH'},
   {'AttributeName': 'codigo', 'KeyType': 'RANGE'}],
  'Projection': {'ProjectionType': 'ALL'},
  'IndexStatus': 'ACTIVE',
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 1,
   'WriteCapacityUnits': 1},
  'IndexSizeBytes': 177,
  'ItemCount': 5,
  'IndexArn': 'arn:aws:dynamodb:sa-east-1:943795379879:table/Clientes/index/nome-codigo-index'}]

Now, we can get Maria's client code by running a quick query!

In [16]:
maria_id = tables['Clientes'].query(Key('nome').eq('Maria'), index=gsi_s[0]['IndexName']).index[0]
maria_id

Decimal('4')